In [1]:
#****************************************************************************
# (C) Cloudera, Inc. 2020-2025
#  All rights reserved.
#
#  Applicable Open Source License: GNU Affero General Public License v3.0
#
#  NOTE: Cloudera open source products are modular software products
#  made up of hundreds of individual components, each of which was
#  individually copyrighted.  Each Cloudera open source product is a
#  collective work under U.S. Copyright Law. Your license to use the
#  collective work is as provided in your written agreement with
#  Cloudera.  Used apart from the collective work, this file is
#  licensed for your use pursuant to the open source license
#  identified above.
#
#  This code is provided to you pursuant a written agreement with
#  (i) Cloudera, Inc. or (ii) a third-party authorized to distribute
#  this code. If you do not have a written agreement with Cloudera nor
#  with an authorized and properly licensed third party, you do not
#  have any rights to access nor to use this code.
#
#  Absent a written agreement with Cloudera, Inc. (“Cloudera”) to the
#  contrary, A) CLOUDERA PROVIDES THIS CODE TO YOU WITHOUT WARRANTIES OF ANY
#  KIND; (B) CLOUDERA DISCLAIMS ANY AND ALL EXPRESS AND IMPLIED
#  WARRANTIES WITH RESPECT TO THIS CODE, INCLUDING BUT NOT LIMITED TO
#  IMPLIED WARRANTIES OF TITLE, NON-INFRINGEMENT, MERCHANTABILITY AND
#  FITNESS FOR A PARTICULAR PURPOSE; (C) CLOUDERA IS NOT LIABLE TO YOU,
#  AND WILL NOT DEFEND, INDEMNIFY, NOR HOLD YOU HARMLESS FOR ANY CLAIMS
#  ARISING FROM OR RELATED TO THE CODE; AND (D)WITH RESPECT TO YOUR EXERCISE
#  OF ANY RIGHTS GRANTED TO YOU FOR THE CODE, CLOUDERA IS NOT LIABLE FOR ANY
#  DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, PUNITIVE OR
#  CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO, DAMAGES
#  RELATED TO LOST REVENUE, LOST PROFITS, LOSS OF INCOME, LOSS OF
#  BUSINESS ADVANTAGE OR UNAVAILABILITY, OR LOSS OR CORRUPTION OF
#  DATA.
#
# #  Author(s): Paul de Fusco
#***************************************************************************/

In [2]:
!pip install --upgrade cdpcli 

In [3]:
# Import inference-related libraries
import httpx
import json
import time
import os
import subprocess
from urllib.parse import urlparse, urlunparse
from typing import Optional, Dict, Any, List

# Used while configuring CDP credentials config
import getpass
from llmopsUtils import *

CDP config function defined!


In [4]:
# Configure CDP control plane credentials
access_key_id = getpass.getpass("Enter your CDP access key ID: ")
private_key = getpass.getpass("Enter your CDP private key: ")
configure_cdp("cdp_access_key_id", access_key_id)
configure_cdp("cdp_private_key", private_key)

Enter your CDP access key ID:  ········
Enter your CDP private key:  ········


In [5]:
ENVIRONMENT_NAME = "pdf-oct-cdp-env"

In [6]:
CAII_DOMAIN = get_caii_domain(ENVIRONMENT_NAME)
CDP_TOKEN = get_ums_jwt_token()
print("CAI DOMAIN: ", CAII_DOMAIN)
print("CDP TOKEN: ", CDP_TOKEN)

CAI DOMAIN:  ml-8c348e69-daf.pdf-oct.a465-9q4k.cloudera.site
CDP TOKEN:  eyJraWQiOiIzYzhlNzA3OTEyZmI0NTA1ODE3NzE3YzMyOTU4MmQwMTFjYjlmNTAwIiwidHlwIjoiSldUIiwiYWxnIjoiUlMyNTYifQ.eyJzdWIiOiJwYXVsZGVmdXNjbyIsImF1ZCI6Imh0dHBzOi8vZGUuYTQ2NS05cTRrLmNsb3VkZXJhLnNpdGUiLCJpc3MiOiJodHRwczovL2NvbnNvbGVhdXRoLmNkcC5jbG91ZGVyYS5jb20vNTU4YmMxZDItODg2Ny00MzU3LTg1MjQtMzExZDUxMjU5MjMzIiwiZ3JvdXBzIjoiY2RwX3NhbmRib3hfd29ya2Vyc193dyBjb21wLWNsdXN0ZXJzLXRlc3RzLWF3LWNkcC1hZG1pbi1ncm91cCBzYW5kYm94LWRlZmF1bHQtc2UtYWRtaW4gY29tcC1jbHVzdGVycy10ZXN0cy1hdy1jZHAtdXNlci1ncm91cCBwYXVsLXByb2pzLW5vdi0xMS1hdy1jZHAtYWRtaW4tZ3JvdXAgcGRmLXNkYngtYXctY2RwLXVzZXItZ3JvdXAgcGRmLXNkYngtYXctY2RwLWFkbWluLWdyb3VwIHBhdWwtcHJvanMtbm92LTExLWF3LWNkcC11c2VyLWdyb3VwIF9jX21sX3VzZXJzXzMxOGUwYjJhIF9jX21sX2FkbWluc181ZDM1NTE4YSBfY19oYmFzZV9hZG1pbnNfNWM0NDExZGMgX2NfbWxfYWRtaW5zX2Y1NTVhYTYgX2NfZGVfdXNlcnNfZGNlNjZhMSBfY19kZV91c2Vyc18yOTNlZWE0NiBfY19oYmFzZV9hZG1pbnNfZGNlNjZhMSBfY19tbF9hZG1pbnNfNDBkMTljMzAgX2NfZGZfdmlld18yOTNlZWE0NjAgX2NfbWxfYWRtaW5z

In [7]:
REGISTRY_ENDPOINT = get_registry_endpoint(ENVIRONMENT_NAME)
print("REGISTRY ENDPOINT: ", REGISTRY_ENDPOINT)

REGISTRY ENDPOINT:  https://modelregistry.ml-29c7521e-a3c.pdf-oct.a465-9q4k.cloudera.site


In [8]:
REGISTERED_MODEL_NAME = "Mixtral-NGC"

In [9]:
model_details = get_model_details(REGISTRY_ENDPOINT, REGISTERED_MODEL_NAME, CDP_TOKEN)
print(model_details)

{'created_at': '2025-11-14T01:06:10.831Z', 'creator': {'user_name': 'pauldefusco'}, 'id': 'lxpn-g4h3-pk8m-tzzu', 'name': 'Mixtral-NGC', 'tags': None, 'updated_at': '2025-11-14T01:06:10.831Z', 'visibility': 'private'}


In [10]:
MODEL_VERSION = get_most_recent_model_version(REGISTRY_ENDPOINT, model_details['id'], CDP_TOKEN)

In [11]:
MODEL_ID = model_details['id']
print(MODEL_ID)
print(MODEL_VERSION)

lxpn-g4h3-pk8m-tzzu
1


In [12]:
ENDPOINT_NAME = "mixtral-endpoint"
deploy_model_to_caii(CAII_DOMAIN, CDP_TOKEN, MODEL_ID, MODEL_VERSION, ENDPOINT_NAME)

Deployed mixtral-endpoint successfully!


In [14]:
# Must return True before we go on to the next step
ready = endpoint_is_ready(CAII_DOMAIN, CDP_TOKEN, ENDPOINT_NAME)
print(ready)

True


In [15]:
BASE_URL = get_endpoint_base_url(CAII_DOMAIN, CDP_TOKEN, ENDPOINT_NAME)
print(BASE_URL)

https://ml-8c348e69-daf.pdf-oct.a465-9q4k.cloudera.site/namespaces/serving-default/endpoints/mixtral-endpoint
